In [2]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
imdb_data = pd.read_csv("IMDB_Dataset.csv")

In [11]:
imdb_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [12]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text.lower())

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

# Apply preprocessing to the 'review' column
imdb_data['preprocessed_review'] = imdb_data['review'].apply(preprocess_text)
imdb_data

,review,sentiment,preprocessed_review
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewer, mentioned, watching, 1, oz, ep..."
1,A wonderful little production. <br /><br />The...,positive,"[wonderful, little, production, br, br, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,negative,"[basically, 's, family, little, boy, jake, thi..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, mattei, 's, ``, love, time, money, ''..."
...,...,...,...
49995,I thought this movie did a down right good job...,positive,"[thought, movie, right, good, job, n't, creati..."
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,I am a Catholic taught in parochial elementary...,negative,"[catholic, taught, parochial, elementary, scho..."
49998,I'm going to have to disagree with the previou...,negative,"['m, going, disagree, previous, comment, side,..."


Word2Vec + LSTM

In [15]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=imdb_data['preprocessed_review'], vector_size=100, window=5, min_count=1, workers=4)

word_embeddings = word2vec_model.wv

# Pad sequences to ensure uniform length
max_len = max(imdb_data['preprocessed_review'].apply(len))
X = imdb_data['preprocessed_review'].apply(lambda tokens: [word_embeddings.key_to_index[word] for word in tokens if word in word_embeddings.key_to_index])
X = pad_sequences(X, maxlen=max_len, padding='post')

# Convert sentiment labels to categorical
y = to_categorical((imdb_data['sentiment'] == 'positive').astype(int))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM model architecture
model = Sequential([
    Embedding(input_dim=len(word_embeddings.key_to_index)+1, output_dim=100, input_length=max_len),
    LSTM(units=128),
    Dropout(0.2),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
563/563 [==============================] - 118s 205ms/step - loss: 0.6936 - accuracy: 0.5035 - val_loss: 0.6932 - val_accuracy: 0.5038
Epoch 2/10
563/563 [==============================] - 76s 135ms/step - loss: 0.6934 - accuracy: 0.5000 - val_loss: 0.6935 - val_accuracy: 0.4963
Epoch 3/10
563/563 [==============================] - 63s 112ms/step - loss: 0.6934 - accuracy: 0.4986 - val_loss: 0.6932 - val_accuracy: 0.4963
Epoch 4/10
563/563 [==============================] - 51s 91ms/step - loss: 0.6932 - accuracy: 0.5042 - val_loss: 0.6936 - val_accuracy: 0.4963
Epoch 5/10
563/563 [==============================] - 49s 87ms/step - loss: 0.6933 - accuracy: 0.5018 - val_loss: 0.6933 - val_accuracy: 0.4963
Epoch 6/10
563/563 [==============================] - 48s 86ms/step - loss: 0.6933 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.5038
Epoch 7/10
563/563 [==============================] - 45s 80ms/step - loss: 0.6933 - accuracy: 0.5005 - val_loss: 0.6941 - val_accur